# EEG Trial assignment
Status: Implemented
 -- Fully automatized

In [1]:
from tqdm.notebook import tqdm
from utils.utils import *

In [2]:
# get all files from participants
folders = get_participant_folder_per_participant()
eeg_file_groups = get_eeg_files_per_participant()
assert (set(folders.keys()) == set(eeg_file_groups.keys()))

### Extract trial epochs
* Check trial assignment 
* extract trial segments
* check for artifacts via voltage violation rules

In [3]:
# Iterate over all participants
for participant in tqdm(folders):
    print('----------------------------------------------')
    print(PARTICIPANT, participant)

    print('(01/09) Load trial annotations')
    trial_annotations_path = get_trial_annotations_path(
        participant, performed=True)
    trial_annotations = pd.read_csv(
        trial_annotations_path, sep=SEPARATOR, dtype={PARTICIPANT: str})

    print('(02/09) Load EEG')
    prepared_eeg, _ = load_eeg_data(
        get_cropped_eeg_path(participant), preload=False)

    print('(03/09) Check trial annotations')
    # check annotation assignment
    check_trial_annotations(trial_annotations, prepared_eeg)

    print('(04/09) Use trial annotations to sync with behavioral data and filter out all unnecessary annotations, rename to snippet name')
    behavioral_data = pd.read_csv(get_behavioral_data_path(
        participant, final_data_exclusion=True), sep=SEPARATOR, dtype={PARTICIPANT: str})
    relevant_trial_annotation_data = get_synchronized_annotations(
        trial_annotations, behavioral_data)

    print('(05/09) Create corresponding dataframe to behavioral data for EEG timestamps')
    # transform to fixation cross, start and end interval per snippet
    eeg_snippet_data = transform_synchronized_annotations(
        relevant_trial_annotation_data)

    print('(06/09) Exclude snippets without EEG data')
    not_provided_snippets = behavioral_data[SNIPPET][~behavioral_data[SNIPPET].isin(
        eeg_snippet_data.index)]
    if not not_provided_snippets.empty:
        print(
            f'Some snippets given in behavioral data do not have an eeg counterpart. {list(not_provided_snippets.values)}')
        update_exclusions(participant, SNIPPET, EEG, {
                          snippet: '01e4_PRE_EEG_Trial_Assignments_Split: No EEG data found' for snippet in not_provided_snippets})

    print('(07/09) Split EEG data into segments for snippets')
    # get eeg data via mne and split it into sections
    # assign each section to the corresponding snippet
    prepared_eeg = prepared_eeg.load_data()
    eeg_segments = split_eeg_segments(prepared_eeg, eeg_snippet_data)

    print('(08/09) Check voltage in segments')
    snippet_violations = check_voltage_per_segment(eeg_segments)
    snippet_violations.to_csv(
        get_eeg_violations_path(participant), sep=SEPARATOR)
    sv = snippet_violations.dropna(axis=0, how='all')
    print(
        f'\tSnippet violations {sv.shape[0]} of {snippet_violations.shape[0]} {round(sv.shape[0]/snippet_violations.shape[0]*100, 2)} %')

    print('(09/09) Save EEG data')
    # save section by snippet name
    for snippet in tqdm(eeg_segments):
        # print(snippet)
        eeg_segment_data: mne.io.Raw = eeg_segments[snippet]
        # print(eeg_segment_data.annotations.to_data_frame())
        eeg_trial_path = get_eeg_trial_path(True, participant, snippet)
        export_eeg_brainvision(eeg_segment_data, eeg_trial_path)

  0%|          | 0/24 [00:00<?, ?it/s]

----------------------------------------------
Participant 003
(01/09) Load trial annotations
(02/09) Load EEG
Extracting parameters from ..\08-Data-Trial_Recordings\processed\003\EEG\EEG_cropped.vhdr...
Setting channel info structure...


(03/09) Check trial annotations
(04/09) Use trial annotations to sync with behavioral data and filter out all unnecessary annotations, rename to snippet name
(05/09) Create corresponding dataframe to behavioral data for EEG timestamps
(06/09) Exclude snippets without EEG data
(07/09) Split EEG data into segments for snippets
Reading 0 ... 1209535  =      0.000 ...  2419.070 secs...
(08/09) Check voltage in segments
	Snippet violations 3 of 72 4.17 %
(09/09) Save EEG data


  0%|          | 0/72 [00:00<?, ?it/s]

Python\Python311\site-packages\pybv\io.py:690: UserWarning: Encountered unsupported non-voltage units: n/a
Note that the BrainVision format specification supports only µV.
  warn(msg)


...

  0%|          | 0/72 [00:00<?, ?it/s]

Python\Python311\site-packages\pybv\io.py:690: UserWarning: Encountered unsupported non-voltage units: n/a
Note that the BrainVision format specification supports only µV.
  warn(msg)


----------------------------------------------
Participant 106
(01/09) Load trial annotations
(02/09) Load EEG
Extracting parameters from ..\08-Data-Trial_Recordings\processed\106\EEG\EEG_cropped.vhdr...
Setting channel info structure...
(03/09) Check trial annotations
(04/09) Use trial annotations to sync with behavioral data and filter out all unnecessary annotations, rename to snippet name
	Annotation Snippet                        12-obf-v1
Time                          462.669968
Stimulus_id                Stimulus/S  3
HDF_index                              0
Participant                          106
onset_e                              0.0
onset            1970-01-01 00:01:30.842
duration                           0.002
description                Stimulus/S  3
onset_float                          1.0
onset_a                              0.0
stimuli_check                       True
Name: 0, dtype: object ignored even though assigned behavioral event, as the behavioral time is expe

  0%|          | 0/72 [00:00<?, ?it/s]

Python\Python311\site-packages\pybv\io.py:690: UserWarning: Encountered unsupported non-voltage units: n/a
Note that the BrainVision format specification supports only µV.
  warn(msg)
